In [1]:
import pyodbc
import pandas as pd
import pyomo.environ as pe
import pyomo.opt as po
model = pe.ConcreteModel("Cargacamiones")
import random
import pandas as pd
random.seed(0)

In [2]:
path=r"C:\Users\jmormal\PycharmProjects\pythonProject3\Carga de camiones\Transporte_cubicaje.mdb"
conn = pyodbc.connect('Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ='+path+';')
cursor = conn.cursor()

Familias Carga de datos

In [3]:
cursor.execute('select IdProducto from Productos')
productos=[]
for row in cursor.fetchall():
    productos.append(row[0])

In [4]:
cursor.execute('select IdFamilia from Familias')
familias=[]
for row in cursor.fetchall():
    familias.append(row[0])

In [5]:
cursor.execute('select IdCamion from Camiones WHERE IdCamion<4')
camiones=[]
for i in range(1,7):
    camiones.append(i)

In [6]:
cursor.execute('select IdDia from Dias WHERE "IdDia"<=19')
dias=[]
for row in cursor.fetchall():
    dias.append(row[0])
        

In [7]:
cursor.execute('select IdDia from Dias')
dias1=[]
for row in cursor.fetchall():
    dias1.append(row[0])



In [8]:
columnas=[1,2]
A1=[6,7]
B1=[4,2]

## Familias Pyomo

In [9]:
model.i = pe.Set(initialize=productos)
model.f = pe.Set(initialize=familias)
model.c = pe.Set(initialize=camiones)
model.t = pe.Set(initialize=dias)
model.t1 = pe.Set(initialize=dias1)
model.col=pe.Set(initialize=columnas)
model.A1=pe.Set(initialize=A1)
model.B1=pe.Set(initialize=B1)

## Carga de datos productos

In [10]:
cursor.execute('select IdFamilia, LotePedido from Familias')
lotepedido={}
for row in cursor.fetchall():
    lotepedido.update({row[0]:row[1]})


In [11]:
cursor.execute('select IdFamilia, LotePedido from Familias')
CosteStock={}
for row in cursor.fetchall():
    CosteStock.update({row[0]:random.randint(7, 10)})
CosteStock

{1: 10,
 2: 10,
 3: 7,
 4: 9,
 5: 10,
 6: 10,
 7: 9,
 8: 10,
 9: 9,
 10: 8,
 11: 8,
 12: 9,
 13: 8,
 14: 7,
 15: 9}

In [12]:
cursor.execute('select IdProducto , PiezasCont from Productos')
piezascont={}
for row in cursor.fetchall():
    piezascont.update({row[0]:row[1]})

In [13]:
cursor.execute('select IdProducto , LargoContedor from Productos')
piezas_largo_box={}
for row in cursor.fetchall():
    piezas_largo_box.update({row[0]:row[1]})

In [14]:
cursor.execute('select IdProducto , IdFamilia, beta from beta')
b={}
for row in cursor.fetchall():
    b.update({(row[0],row[1]):row[2]})


In [15]:
cursor.execute('select  IdProducto , IdDia, Demanda from Demanda')
demanda={}
for row in cursor.fetchall():
    # if row[0]>=13:
    #     demanda.update({(row[0],row[1]):row[2]})
    # else:
        demanda.update({(row[0],row[1]):row[2]})    

In [16]:
# cursor.execute('select IdProducto , StockInicial from Productos')
stockInicial={}
# for row in cursor.fetchall():
#     stockInicial.update({row[0]:row[1]})
cursor.execute('select  IdProducto , Demanda from Demanda where IdDia=1')

for row in cursor.fetchall():
    stockInicial.update({row[0]:row[1]*3})


In [17]:
stockInicial

{1: 2025,
 3: 1092,
 5: 435,
 2: 2037,
 4: 810,
 6: 681,
 7: 315,
 8: 453,
 9: 96,
 10: 81,
 11: 87,
 12: 84,
 13: 2052,
 14: 81,
 15: 2016}

In [18]:
cursor.execute('select  IdProducto , Valor from Productos')
valor={}
for row in cursor.fetchall():
    valor.update({row[0]:row[1]})

In [19]:
PC={}
for camion in camiones:
    PC.update({camion:1000})

In [20]:
L=78
nu=26/30
NT_lw=5
NT_up=10

In [21]:
model.LotePedido = pe.Param(model.f , initialize=lotepedido)
model.PiezasCont= pe.Param(model.i ,initialize=piezascont)
model.b= pe.Param(model.i, model.f ,initialize=b, default=0)
model.StockInicial= pe.Param(model.i, initialize=stockInicial)
model.demanda= pe.Param(model.i, model.t1,  initialize=demanda)
model.valor=pe.Param(model.i, initialize=stockInicial)
model.PC=pe.Param(model.c, initialize=PC)
model.L=pe.Param(initialize=L)
model.nu=pe.Param(initialize=nu)
model.NT_lw=pe.Param(initialize=NT_lw)
model.NT_up=pe.Param(initialize=NT_up)
model.LargoCont=pe.Param(model.i,initialize=piezas_largo_box)
model.CS=pe.Param(model.i, initialize=CosteStock)

    'Any'. The default domain for Param objects is 'Any'.  However, we will be
    changing that default to 'Reals' in the future.  If you really intend the
    domain of this Paramto be 'Any', you can suppress this warning by
    explicitly specifying 'within=Any' to the Param constructor.  (deprecated
    in 5.6.9, will be removed in (or after) 6.0) (called from
    C:\Users\jmormal\Anaconda3\envs\cubicaje38\lib\site-
    packages\pyomo\core\base\indexed_component.py:645)


# # Varibles

In [22]:
model.CantidadPedir = pe.Var(model.i, model.c, model.t,model.col, domain=pe.Integers)

In [23]:
model.CantidadPedir2 = pe.Var(model.i,model.f, model.c, model.t,model.col, domain=pe.Integers, bounds=(0,None) )
model.ContenedoresCamion=pe.Var(model.c,model.t,model.col,model.i, domain=pe.Integers)

In [24]:
model.Stock=pe.Var(model.i,model.t, bounds=(0,None))

In [25]:
model.RetrasoDemanda=pe.Var(model.i,model.t, bounds=(0,None))

In [26]:
model.K=pe.Var(model.f,model.c,model.t,model.col ,domain=pe.Integers)

In [27]:
model.Y=pe.Var(model.c,model.t ,domain=pe.Binary, initialize=0)

In [28]:
# model.A=pe.Var(model.c,model.col,model.t, domain=pe.Integers, bounds=(0,7.5))
# model.B=pe.Var(model.c,model.col, model.t, domain=pe.Integers, bounds=(0,4.5))
model.A=pe.Var(model.c,model.col,model.t, domain=pe.Integers, bounds=(0,13))
model.B=pe.Var(model.c,model.col, model.t, domain=pe.Integers, bounds=(0,13))


## Función Objetivo
Minimizar el número de camiones usado
$$\sum_{c,t}PC(c)Y(c,t)+ \sum CS(i)Stock(i,t)$$

In [29]:
expr = sum(model.PC[c]*model.Y[c,t] +sum(model.CS[i]*model.Stock[i,t] for i in model.i) for c in model.c for t in model.t)

In [30]:
 #Coste Transporte
model.objective = pe.Objective(sense=pe.minimize, expr=expr)

## Funciones auxiliares

In [31]:
def UsoCamion(model):
    return sum(model.L*model.Y[c,t] for c in model.c for t in model.t)-\
        sum(model.ContenedoresCamion[c,t] for c in model.c for t in model.t)

In [32]:
def CTransporte(model):
    return sum(model.PC[c]*model.Y[c,t] for c in model.c for t in model.t)

In [33]:
def NumeroCamiones(model):
    return sum(model.Y[c,t] for c in model.c for t in model.t)

## Restricciones

In [34]:
model.stocks=pe.ConstraintList()

$$Stock(i,t)=Stock(i,t-1)+\sum_{c,col}CantidadPedir(i,c,t,col)PiezasCont[i] -Demanda(i,t)$$

In [35]:
for i in model.i:
        lhs=model.Stock[i,1]
        rhs=model.StockInicial[i]-model.demanda[i,1]+sum(model.CantidadPedir[i,c,1,col]*model.PiezasCont[i] for c in model.c for col in model.col )
        model.stocks.add(lhs==rhs)

In [36]:
for t in list(model.t)[1:]:
    for i in model.i:
            lhs=model.Stock[i,t]
            rhs=model.Stock[i,t-1]-model.demanda[i,t-1]+sum(model.CantidadPedir[i,c,t-1,col]*model.PiezasCont[i] for c in model.c for col in model.col)
            model.stocks.add(lhs==rhs)

In [37]:
model.RcantidadPedir=pe.ConstraintList()

$$CantidadPedir2_{i,f,c,t,col}=K(f,c,t,col)*LotePedido(f)*b(i,f)$$

In [38]:
for i in model.i:
    for f in model.f:
        for c in model.c:
            for t in model.t:
                for col in model.col:
                    lhs=model.CantidadPedir2[i,f,c,t,col]
                    rhs=model.K[f,c,t,col]*model.LotePedido[f]*model.b[i,f]
                    model.RcantidadPedir.add(lhs==rhs)

In [39]:
model.RcantidadPedir2=pe.ConstraintList()

$$CantidadPedir_{i,c,t,col}= \sum_f CanitdadPedir2(i,f,c,t,col)$$

In [40]:
for i in model.i:
    for c in model.c:
        for t in model.t:
            for col in model.col:
                lhs=model.CantidadPedir[i,c,t,col]
                rhs=sum(model.CantidadPedir2[i,f,c,t,col] for f in model.f)
                model.RcantidadPedir2.add(lhs==rhs)

In [41]:
model.RDimensionesCamion=pe.ConstraintList()

$$ 1600A_{c,col,t} +1000B_{c,col,t}   \leq 13600* Y[c,t]$$

In [42]:
for c in model.c:
    for t in model.t:
        for col in model.col:
            lhs=13600*model.Y[c,t]
            rhs=(model.A[c,col,t]*1600+model.B[c,col,t]*1000)
            model.RDimensionesCamion.add(lhs>=rhs)

In [43]:
model.RCajasTotales=pe.ConstraintList()

$$ \forall c \in C \quad \forall t \in T \quad \sum_i CantidadPedir(i,c,t,col) \leq A_{c,col,t}*4+B_{c,col,t}*3 $$

In [44]:
for c in model.c:
    for t in model.t:
        for col in model.col:
            lhs=sum(model.CantidadPedir[i,c,t,col] for i in model.i)
            rhs=(model.A[c,col,t]*4+model.B[c,col,t]*3)
            model.RCajasTotales.add(lhs<=rhs)

In [45]:
model.RCajasTipoA=pe.ConstraintList()

$$ \sum_i CantidadPedir(i,c,t,col)_{tipo a}\leq A_{c,col,t}*4 $$

In [46]:
for c in model.c:
    for t in model.t:
        for col in model.col:
            lhs=sum(model.CantidadPedir[i,c,t,col]*float(int(model.LargoCont[i])==int(1600)) for i in model.i)
            rhs=model.A[c,col,t]*4
            model.RCajasTipoA.add(lhs<=rhs)

$$ \sum_i CantidadPedir(i,c,t,col)_{tipo b}\leq B_{c,col,t}*3 $$

In [47]:
model.RCajasTipoB=pe.ConstraintList()

In [48]:
for c in model.c:
    for t in model.t:
        for col in model.col:
            lhs=sum(model.ContenedoresCamion[c,t,col,i]*float(int(model.LargoCont[i])==int(1000)) for i in model.i)
            rhs=model.B[c,col,t]*3
            model.RCajasTipoB.add(lhs<=rhs)

In [49]:
model.RColumnasCamion=pe.ConstraintList()
for c in model.c:
     model.RColumnasCamion.add(model.A[c,2,t]== model.A[c,1,t])
     model.RColumnasCamion.add(model.B[c,2,t]== model.B[c,1,t])


$$\forall i,t  \quad Stock[i,t]\geq \sum_{j \in [1,2,3,4]} Demanda[i,t+j]$$

In [50]:
model.RCobertura=pe.ConstraintList()
l=list(range(25))
for i in model.i:
    for t in range(1,17):
        lhs=model.Stock[i,t]
        rhs=(model.demanda[i,t+1]+model.demanda[i,t+2]+model.demanda[i,t+3])
        model.RCobertura.add(lhs>=rhs)
        l[i]=rhs
for i in model.i:
    for t in range(17,20):
        lhs=model.Stock[i,t]
        rhs=l[i]
        model.RCobertura.add(lhs>=rhs)


In [51]:
model.RCVolumenminimo=pe.ConstraintList()

In [52]:
for t in model.t:
    for c in model.c:
        lhs=(sum(model.CantidadPedir[i,c,t,col] for i in model.i for col in model.col if int(model.LargoCont[i])==int(1600))*750*1200*1600+
                                     +(sum(model.CantidadPedir[i,c,t,col] for i in model.i for col in model.col if int(model.LargoCont[i])==int(1000)))*1000*1000*1200)/(3000*2400*13600)
        rhs=0.94*model.Y[c,t]
        model.RCVolumenminimo.add(lhs >= rhs)

In [53]:
model.bounds=pe.ConstraintList()
for i in model.i:
    for t in model.t:
        model.bounds.add(model.Stock[i,t]>=0)


In [54]:
# solver = po.SolverFactory("gurobi")
# solver.options["TimeLimit"]=60
# # solver.options["MIPFocus"] = 0
# solver.options["MIPGap"] = 0.01
# results = solver.solve(model, tee=True)


solvername='cbc'
solverpath_exe=r"C:\Users\jmormal\Downloads\cbc-win64\cbc.exe"
# solverpath_exe=r"C:\Users\jmormal\Downloads\Cbc-2.10-x86_64-w64-mingw32-dbg (1)\bin"

solver = po.SolverFactory(solvername,executable=solverpath_exe)
# solver.options["TimeLimit"]=60
# # solver.options["MIPFocus"] = 0
# solver.options["MIPGap"] = 0.01
solver.options = {'sec': 100}
results = solver.solve(model,tee=True)

Welcome to the CBC MILP Solver 
Version: 2.10.5 
Build Date: Nov 24 2021 

command line - C:\Users\jmormal\Downloads\cbc-win64\cbc.exe -sec 100 -printingOptions all -import C:\Users\jmormal\AppData\Local\Temp\13\tmp1zh0ywp3.pyomo.lp -stat=1 -solve -solu C:\Users\jmormal\AppData\Local\Temp\13\tmp1zh0ywp3.pyomo.soln (default strategy 1)
seconds was changed from 1e+100 to 100
Option for printingOptions changed from normal to all
Presolve is modifying 180 integer bounds and re-presolving
Presolve 1077 (-55537) rows, 4263 (-55417) columns and 15015 (-110965) elements
Statistics for presolved model
Original problem has 59394 integers (114 of which binary)
Presolved problem has 3978 integers (114 of which binary)
==== 3864 zero objective 6 different
3864 variables have objective of 0
38 variables have objective of 42
57 variables have objective of 48
95 variables have objective of 54
95 variables have objective of 60
114 variables have objective of 1000
==== absolute objective values 6 differ

In [55]:
def mean(x):
    return sum(x)/len(x)

In [56]:
i=14
for t in range(1,16):
    print(t,model.Stock[i,t].value,-model.demanda[i,t+1]-model.demanda[i,t+2]-model.demanda[i,t+3] ,
          model.Stock[i,t].value-model.demanda[i,t+1]-model.demanda[i,t+2]-model.demanda[i,t+3])
    a=-model.demanda[i,t+1]-model.demanda[i,t+2]-model.demanda[i,t+3]-model.demanda[i,t+4]
for t in range(16,20):
    print(t,model.Stock[i,t].value,a ,model.Stock[i,t].value+a)
a
i

1 77.5 -73 4.5
2 74.0 -74 0.0
3 78.0 -78 0.0
4 83.0 -83 0.0
5 83.0 -83 0.0
6 79.0 -79 0.0
7 76.0 -76 0.0
8 74.0 -74 0.0
9 75.0 -75 0.0
10 76.0 -76 0.0
11 77.0 -77 0.0
12 79.0 -79 0.0
13 77.0 -77 0.0
14 79.0 -79 0.0
15 77.0 -77 0.0
16 79.0 -104 -25.0
17 79.0 -104 -25.0
18 79.0 -104 -25.0
19 79.0 -104 -25.0


14

In [57]:
utilizacion=0
volumen_por_camion=[]
for t in model.t:
    for c in model.c:
        if model.Y[c,t].value==1:
            volumen_por_camion.append(((sum(model.CantidadPedir[i,c,t,col].value for i in model.i for col in model.col if int(model.LargoCont[i])==int(1600))*750*1200*1600+
                                     +(sum(model.CantidadPedir[i,c,t,col].value for i in model.i for col in model.col if int(model.LargoCont[i])==int(1000)))*1000*1000*1200)/
                                     (3000*2400*13600)))
volumen_por_camion
print(mean(volumen_por_camion))
print(len(volumen_por_camion))

0.9999999980042017
7


In [58]:
for t in model.t:
    print(sum(model.Y[c,t].value for c in model.c))

2.6155600100000003
2.452071269
2.552919816
2.7047497
2.5597890560000005
2.64965928
2.44905675
2.6201193910000002
2.66376541
2.6843508670000005
2.5381833119999997
2.59446486
2.6418662790000003
2.608454954
2.580411664
2.62848615
2.603962421
2.627968102
0.0


In [59]:
l=[]
for t in model.t:
    for i in model.i:
        1
        # print(t,i,model.Stock[i,t].value, sum(model.Y[c,t].value  for c in model.c if model.CantidadPedir[i,c,t,col].value>0))

            # if model.Y[c,t].value==1:
            #     print(t,model.A[c,col,t].value,model.B[c,col,t].value,((sum(model.CantidadPedir[i,c,t,col].value/model.PiezasCont[i] for i in model.i for col in model.col if int(model.LargoCont[i])==int(1600))*750*1200*1600+
            #                          +(sum(model.CantidadPedir[i,c,t,col].value/model.PiezasCont[i] for i in model.i for col in model.col if int(model.LargoCont[i])==int(1000)))*1000*1000*1200)/
            #                          (3000*2400*13600)))

In [60]:
l=[]
for t in model.t:
    for c in model.c:
        

            if model.Y[c,t].value==1:
                l.append([t,model.A[c,col,t].value,model.B[c,col,t].value,((sum(model.CantidadPedir[i,c,t,col].value for i in model.i for col in model.col if int(model.LargoCont[i])==int(1600))*750*1200*1600+
                                     +(sum(model.CantidadPedir[i,c,t,col].value for i in model.i for col in model.col if int(model.LargoCont[i])==int(1000)))*1000*1000*1200)/
                                     (3000*2400*13600),[model.CantidadPedir[i,c,t,col].value for i in model.i])])
                print([model.CantidadPedir[i,c,t,col].value for i in model.i])
                # print(t,model.A[c,col,t].value,model.B[c,col,t].value,((sum(model.CantidadPedir[i,c,t,col].value for i in model.i for col in model.col if int(model.LargoCont[i])==int(1600))*750*1200*1600+
                #                      +(sum(model.CantidadPedir[i,c,t,col].value for i in model.i for col in model.col if int(model.LargoCont[i])==int(1000)))*1000*1000*1200)/
                #                      (3000*2400*13600)),
                #     sum(model.CantidadPedir[i,c,t,col].value for i in model.i for col in model.col if int(model.LargoCont[i])==int(1600))
                #       ,sum(model.CantidadPedir[i,c,t,col].value for i in model.i for col in model.col if int(model.LargoCont[i])==int(1000)))

[0.0, 19.366667, 14.633333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[8.452399, 0.0, 0.0, 0.0, 8.002399, 0.0, 0.0, 0.0, 0.0, 0.0, 2.2666667, 0.0, 0.0, 1.0909091, 17.243333]
[0.0, 0.0, 22.933333, 0.0, 0.0, 7.7116162, 0.0, 0.0, 0.0, 0.0, 0.0, 1.8666667, 0.0, 0.0, 1.7860606]
[0.0, 0.0, 0.0, 0.0, 7.5787879, 0.0, 7.6, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 22.585455]
[0.0, 0.0, 0.0, 2.3451389, 0.0, 7.9427399, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 28.454545]
[0.0, 16.95, 17.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 27.333333, 0.0, 0.0, 0.0, 6.6666667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [61]:
df1=pd.DataFrame(l)
df1

,0,1,2,3
0,2,8.500000,0.000000,"(0.9999999955882352, [0.0, 19.366667, 14.63333..."
1,4,4.680366,6.111414,"(1.0000000051470588, [8.452399, 0.0, 0.0, 0.0,..."
2,7,8.127904,0.595354,"(0.9999999985294119, [0.0, 0.0, 22.933333, 0.0..."
3,11,3.794697,7.528485,"(1.0000000014705883, [0.0, 0.0, 0.0, 0.0, 7.57..."
4,12,2.571970,9.484849,"(0.9999999946078432, [0.0, 0.0, 0.0, 2.3451389..."
5,15,8.500000,0.000000,"(0.9999999926470589, [0.0, 16.95, 17.05, 0.0, ..."
6,17,8.500000,0.000000,"(0.9999999980392157, [0.0, 0.0, 27.333333, 0.0..."


In [62]:
l1=[]
a=list(range(25))
for t in range(1,17):
    for i in model.i:
            l1.append([t,i,model.Stock[i,t].value,-model.demanda[i,t+1]-model.demanda[i,t+2]-model.demanda[i,t+3] ,
          model.Stock[i,t].value-model.demanda[i,t+1]-model.demanda[i,t+2]-model.demanda[i,t+3]])
            a[i]=-model.demanda[i,t+1]-model.demanda[i,t+2]-model.demanda[i,t+3]
for t in range(17,20):
    for i in model.i:
            l1.append([t,i,model.Stock[i,t].value,a[i] ,
          model.Stock[i,t].value+a[i]])
a

[0,
 -1812,
 -1905,
 -1161,
 -753,
 -429,
 -713,
 -330,
 -457,
 -90,
 -96,
 -89,
 -92,
 -1968,
 -79,
 -1860,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24]

In [63]:
df2=pd.DataFrame(l1)
pd.set_option('display.max_rows', None)
df2

,0,1,2,3,4
0,1,1,1983.5,-1945,38.5
1,1,2,1911.0,-1831,80.0
2,1,3,1091.5,-1080,11.5
3,1,4,775.5,-754,21.5
4,1,5,429.0,-427,2.0
5,1,6,739.0,-739,0.0
6,1,7,323.0,-323,0.0
7,1,8,439.0,-427,12.0
8,1,9,94.5,-89,5.5
9,1,10,89.0,-89,0.0


In [64]:
0.8628883539414994
79

79

In [65]:
for t in model.t:
    print(sum(model.CantidadPedir[i,c,t,col].value for c in model.c for i in model.i for col in model.col if int(model.LargoCont[i])==int(1000)))
sum(model.CantidadPedir[i,c,t,col].value for t in model.t for c in model.c for i in model.i for col in model.col if int(model.LargoCont[i])==int(1000))

44.969696299999995
45.58901438
42.1382578
48.0075761
40.378786999999996
45.0321971
42.8030305
48.36174213
45.7632576
42.8030306
45.0757581
43.2083325
44.3162881
43.92992429999999
43.221590809999995
43.384469900000006
43.58333331
44.7821966
0.0


797.3484831300002

In [66]:
sum(model.CantidadPedir[i,c,t,col].value for c in model.c for t in model.t for i in model.i for col in model.col if int(model.LargoCont[i])==int(1600))

2516.2999985249985

In [67]:
sum(model.Demanda for c in model.c for t in model.t for i in model.i if int(model.LargoCont[i])==int(1600))

AttributeError: 'ConcreteModel' object has no attribute 'Demanda'

In [ ]:
for i,v in model.LargoCont:
    print(i)

In [ ]:
sum(model.demanda[i,t]/ for )

In [ ]:
import scip